<a href="https://colab.research.google.com/github/BhekiMabheka/Explore/blob/main/Employee_Addmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective:**
- Train a model that would predict if a candidate will be hired

In [1]:
# Data Wrangling Packages
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import warnings

# Machine Learning Packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import *

template = 'plotly_dark'
warnings.filterwarnings("ignore")

In [3]:
admissions_df = pd.read_csv("https://raw.githubusercontent.com/BhekiMabheka/Data/master/admissions.csv")

In [4]:
# Num of obeservations and features
print("Num of rows and features: ", admissions_df.shape) 
admissions_df.head(1)

Num of rows and features:  (64722, 3)


,candidate_id,department,admit
0,933951,E,Rejected


In [5]:
candinates_df = pd.read_csv("https://raw.githubusercontent.com/BhekiMabheka/Data/master/candidates.csv")

In [6]:
# Num of obeservations and features
print("Num of rows and features: ", candinates_df.shape) 
candinates_df.head(3)

Num of rows and features:  (49786, 6)


,name,candidate_id,age,gender,no_children,emp_card_id
0,Geneva Le,928100,43.0,Female,NaN,7505.0
1,Marilyn Weber,913797,51.0,Male,0.0,NaN
2,Marlene Deweese,926775,30.0,Male,2.0,NaN


In [7]:
# Join admissions and candinates datasets
df  = pd.merge(left = admissions_df, left_on = ['candidate_id'], right = candinates_df, right_on = ['candidate_id'], how = 'inner')
print("Num of obeservations and features: ", df.shape)
df.head(3)

Num of obeservations and features:  (64722, 8)


,candidate_id,department,admit,name,age,gender,no_children,emp_card_id
0,933951,E,Rejected,Chester Joshua,43.0,Male,2.0,NaN
1,933951,E,Rejected,Chester Joshua,43.0,Male,2.0,NaN
2,910365,B,Admitted,Ralph Turner,44.0,Male,0.0,1010.0


In [8]:
# Check the unique entries on categorical features
def check_unique_entries(df):
    for col in df.select_dtypes(include=['bool', 'object']).columns.tolist():
        print(col," :",df[col].unique())
        
check_unique_entries(df)

department  : ['E' 'B' 'D' 'A' 'F' 'C']
admit  : ['Rejected' 'Admitted']
name  : ['Chester Joshua' 'Ralph Turner' 'Chelsea Muszynski' ... 'Maria Radford'
 'Mary Greggs' 'Justin Cook']
gender  : ['Male' 'Female']


In [9]:
def explore_df(df):   
    """
    A more advanced version of describe for tabular exploratory data analysis. Inlcudes additional information such as,
    missing observations, unique observations, constant feature flagging, all_missing feature flagging, feature types & outlier
    values.
    Parameters
    ----------
    df : pandas df, required, default=NA
        Pandas dataframe object 
    Returns
    -------
    pandas df
        Returns a pandas dataframe object
    
    Usage
    -----
        df = pd.DataFrame({"x1": ["a", "b", "c", "a"], "x2":['x','y','x','x'], "y": [1,1,0,1]})
        eda = explore_df(df=df)
    """
    import pandas as pd
    import numpy as np
    
    ft                     = pd.DataFrame()
    ft['type']             = df.dtypes.astype(str)
    ft['feature']          = ft.index
    ft['num_of_unique']    = df.nunique()
    ft['num_of_missing']   = df.isnull().sum()
    ft['num_of_missing_%'] = (df.isnull().sum()/ df.isnull().count()).round(2)
    
    numeric = ft.loc[(ft['type'].str.contains('float'))]['feature']
    numeric = numeric.append(ft.loc[(ft['type'].str.contains('int'))]['feature'])
    
    categorical = ft.loc[(ft['type'].str.contains('object'))]['feature']

    # Summary statistics
    lower=df[numeric].quantile(q=0.25)
    upper=df[numeric].quantile(q=0.75)
    ft['min']=df[numeric].min()
    ft['max']=df[numeric].max()
    ft['q1']=lower
    ft['median']=df[numeric].median()
    ft['mean']=df[numeric].mean()
    ft['q3']=upper
    
    # Caclulate outlier values
    iqr = upper - lower
    lower=lower-(1.5*iqr)
    upper=upper+(1.5*iqr)
    ft['lower_outlier']=lower
    ft['upper_outlier']=upper
    ft['skewness']=df[numeric].skew()
        
    ft=ft[['feature','type','num_of_missing','num_of_missing_%','num_of_unique','min','q1','median','mean','q3','max','lower_outlier',
           'upper_outlier','skewness']].reset_index(drop=True)

    return ft

explore_df(df)

,feature,type,num_of_missing,num_of_missing_%,num_of_unique,min,q1,median,mean,q3,max,lower_outlier,upper_outlier,skewness
0,candidate_id,int64,0,0.00,49786,896465.0,908888.0,921393.5,921365.843114,933816.75,9.462500e+05,871494.875,971209.875,-0.000859
1,department,object,0,0.00,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,admit,object,0,0.00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,name,object,0,0.00,47415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,age,float64,0,0.00,59,11.0,33.0,39.0,66399.551111,46.00,2.147484e+09,13.500,65.500,179.887465
5,gender,object,0,0.00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,no_children,float64,9552,0.15,5,0.0,0.0,1.0,1.052075,2.00,4.000000e+00,-3.000,5.000,0.201209
7,emp_card_id,float64,42124,0.65,8276,0.0,2506.0,5054.5,5011.285866,7516.00,9.999000e+03,-5009.000,15031.000,-0.014461


---
#### Comments on `data quality`.

- Out of 64 722 candinates 65%(42 124) don't have `employee card id` and it looks there's some un-trusted entries since the max_value it's 9999
- Out of 64 722 candinates 15% don't have children or their number of children it's not captured/reported. So from Machine Learning perspective it won't be safe to impute the missings values with zero. In other words having to impute with zero, an employee might lose certain benefits. We rather investigate or delete the entries!
- It looks like we do have outlier on a feature `age`, no one can live up to this `2147483647`
- Again of the feature `age` we cannot compute an average with outliers-this means lower and upper outliers should be handled.
- It's interesting how can a candinate start to work/apply at the `age of 11` - this sounds like a child labour!
- There's about six departments


----
